In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks

In [2]:
# 1. 데이터 로딩: BERT만 사용
def load_data(path):
    df = pd.read_json(path)
    bert = np.array(df['bert_cls'].tolist())
    y = df['fake'].values
    return bert, y

In [3]:
# 2. 모델 정의: BERT만 사용
def build_model():
    bert_input = layers.Input(shape=(768,))

    x = layers.Dense(256, activation='relu')(bert_input)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=bert_input, outputs=output)
    model.compile(optimizer=optimizers.Adam(1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [4]:
# 3. 학습 함수 수정: 입력 하나만
def train_model(X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42)

    model = build_model()

    early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=64,
        callbacks=[early_stop]
    )

    y_pred_prob = model.predict(X_val)
    y_pred = (y_pred_prob > 0.5).astype(int)

    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1 Score : {f1:.4f}")

    return model

In [17]:
# # 실행
X, y = load_data("emb_cls.json")
model = train_model(X, y)


Epoch 1/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6745 - loss: 0.6014 - val_accuracy: 0.7208 - val_loss: 0.5475
Epoch 2/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7176 - loss: 0.5520 - val_accuracy: 0.7226 - val_loss: 0.5412
Epoch 3/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7212 - loss: 0.5455 - val_accuracy: 0.7259 - val_loss: 0.5378
Epoch 4/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7256 - loss: 0.5380 - val_accuracy: 0.7265 - val_loss: 0.5364
Epoch 5/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7297 - loss: 0.5304 - val_accuracy: 0.7244 - val_loss: 0.5364
Epoch 6/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7329 - loss: 0.5286 - val_accuracy: 0.7299 - val_loss: 0.5321
Epoch 7/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7334 - loss: 0.5295 - val_accuracy: 0.7283 - val_loss: 0.5310
Epoch 8/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7349 - loss: 0